In [1]:
# Install required libraries
!pip install pathway bokeh panel --quiet


In [2]:
import pandas as pd
import pathway as pw
from pathway.internals.join_mode import JoinMode
from datetime import datetime
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
import panel as pn


In [3]:
# Load dataset and combine date/time
df = pd.read_csv("/content/dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

def vehicle_type_weight(v):
    return {'car': 1.0, 'bike': 0.5, 'truck': 1.5}.get(str(v).lower(), 1.0)



In [4]:
df['VehicleTypeWeight'] = df['VehicleType'].apply(vehicle_type_weight)

# Rename column to match expected schema
df.rename(columns={"TrafficConditionNearby": "TrafficCongestion"}, inplace=True)

# Save required features
df[['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficCongestion', 'IsSpecialDay', 'VehicleTypeWeight', 'Latitude', 'Longitude']].to_csv("parking_stream.csv", index=False)




In [5]:
# Define schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficCongestion: float
    IsSpecialDay: int
    VehicleTypeWeight: float
    Latitude: float
    Longitude: float

In [6]:
# Replay CSV as stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

# Add time features
fmt = "%Y-%m-%d %H:%M:%S"
data = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [7]:
# Base model: Linear Pricing (Model 1)
model1 = data.with_columns(
    price_linear = 10 + (data.Occupancy / data.Capacity) * 5  # alpha = 5
)

In [8]:
#remove this model 2 based earlier code

@pw.udf
def normalize_demand(o, q, tc, sd, vw, cap):
    d = 1.0 * (o / cap) + 0.05 * q - 0.1 * tc + 0.5 * sd + 0.2 * vw
    return max(0, min(d, 1))

@pw.udf
def compute_price_from_demand(d):
    return max(5, min(10 * (1 + d), 20))

model2_intermediate = data.with_columns(
    norm_demand = normalize_demand(
        pw.this.Occupancy,
        pw.this.QueueLength,
        pw.this.TrafficCongestion,
        pw.this.IsSpecialDay,
        pw.this.VehicleTypeWeight,
        pw.this.Capacity
    )
)

model2 = model2_intermediate.with_columns(
    price_demand = compute_price_from_demand(pw.this.norm_demand)
)


In [9]:
# Model 2: Demand-based final code
def clamp(x, min_val=5, max_val=20):
    return pw.apply(lambda v: max(min_val, min(max_val, v)), x)

model2 = data.with_columns(
    demand = (
        (data.Occupancy / data.Capacity) +
        0.3 * data.QueueLength +
        0.2 * data.TrafficCongestion +
        0.2 * data.IsSpecialDay
    ),
    price_model2 = clamp(10 * (1 + (data.Occupancy / data.Capacity)))
)

In [10]:
# Model 3: Competitive Pricing (with rerouting suggestion)
@pw.udf
def reroute_suggestion(occ, cap, my_price, near_price):
    if occ / cap > 0.95 and my_price > near_price:
        return "Suggest reroute"
    return "No reroute"


In [11]:
model2_copy = model2.rename_columns(
    Timestamp_right="Timestamp",
    Latitude_right="Latitude",
    Longitude_right="Longitude",
    price_model2_right="price_model2"
)

joined = model2.join(model2_copy, how=JoinMode.INNER)

# Filter out self-joins (same time and location)
filtered = joined.filter(
    (joined["Timestamp"] != joined["Timestamp_right"]) &
    (joined["Latitude"] != joined["Latitude_right"]) &
    (joined["Longitude"] != joined["Longitude_right"])
)

In [12]:
# ---------------- Haversine Distance Function ----------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1_rad = np.radians(lat1)
    lat2_rad = np.radians(lat2)
    delta_lat = np.radians(lat2 - lat1)
    delta_lon = np.radians(lon2 - lon1)

    a = np.sin(delta_lat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Note: This is a pure Python function, so Pathway cannot use it directly.
# You need to implement rerouting using precomputed distances or integrate via Pandas/UDF.

# Select some useful outputs from the competitive model (optional for debugging or rerouting logic)
filtered.select(
    timestamp1 = joined["Timestamp"],
    timestamp2 = joined["Timestamp_right"],
    lat1 = joined["Latitude"],
    lat2 = joined["Latitude_right"],
    long1 = joined["Longitude"],
    long2 = joined["Longitude_right"],
    price1 = joined["price_model2"],
    price2 = joined["price_model2_right"]
).show()

# Visualization with Bokeh and real-time plotting can be added as a next step.


Column
    [0] Row
        [0] Markdown(str)
        [1] TooltipIcon(value='Table depends o...)
    [1] Tabulator(disabled=True, height=400, page_size=10, pagination='local', show_index=False, sizing_mode='stretch_width', value=Empty DataFrame
Columns: [...)

In [13]:
pn.extension()

source = ColumnDataSource(data={
    'x': df['Timestamp'],
    'y': 10 + (df['Occupancy'] / df['Capacity'])  # Using Model 1 price
})

plot = figure(x_axis_type="datetime", title="Dynamic Parking Prices Over Time", width=800, height=400)
plot.line(x='x', y='y', source=source, line_width=2, color='navy')
plot.circle(x='x', y='y', source=source, size=6, color='red')

panel = pn.Column("# Dynamic Price Visualization", plot)
panel.servable()


Column
    [0] Markdown(str)
    [1] Bokeh(figure)